# Gemini: Una descripción general de los escenarios multimodales

## Visión General

### Gemini

Gemini es una familia de modelos generativos de IA desarrollados por Google DeepMind y diseñados para casos de uso multimodales. La API Gemini da acceso a los modelos Gemini Pro y Gemini Pro Vision

### API Vertex AI Gemini

La API Vertex AI Gemini nos da una interfaz unificada para interactuar con modelos Gemini. Actualmente existen dos modelos disponibles en la API Gemini:

- **Modelos Gemini Pro** (`gemini-pro`): Diseñado para manejar tareas de lenguaje natural, chat multiturno de texto y código y generación de código.
- **Modelo Gemini Pro Vision** (`gemini-pro-vision`): Soporta prompts multimodales. Puedes incluir texto, imágenes y video en los prompt y obtener respuestas en texto o código.

Puedes interactuar con la API Gemini usando los siguientes métodos:

- Usando [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para pruebas rápidas y generación de contenidos de texto.
- Usando el SDK de Vertex AI

Esta libreta se concentra en el uso de **Vertex AI SDK para Python** para utilizar la API Vertex AI Gemini.

Para obtener más información, consulta la documentación [IA Generativa en Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

Esta libreta demuestra una variedad de casos de uso multimodal para los que se puede utilizar Gemini.

#### Casos de uso multimodales

En comparación con los LLM (*Large Language Model*, modelos grandes de lenguaje) de sólo texto, la multimodalidad de Gemini Pro Vision se puede utilizar para muchos casos de uso nuevos:

Ejemplos de casos de uso con **texto e imágenes** como entrada:

- Detectar objetos en fotografías
- Comprender las pantallas y las interfaces
- Compresión de dibujos y abstracciones
- Comprender gráficos y diagramas
- Recomendación de imágenes basada en las preferencias del usuario
- Comparar imágenes en busca de similitudes, anomalías o diferencias

Ejemplos de casos de uso con **texto y video** como entrada:

- Generar la descripción de un video
- Extraer etiquetas de objetos a lo largo de un video
- Extraer momentos destacados/mensajes de un video

### Costos

Este tutorial usa los siguientes componentes de Google Cloud que pueden generar costos en su factura:

- Vertex AI

Para mayores detalles puedes revisar los [precios de Vertex AI](https://cloud.google.com/vertex-ai/pricing) y usar la [calculadora de precios](https://cloud.google.com/products/calculator/) para generar una estimación de costos con base en el uso del proyecto.

### **Sólo para uso en Colab - Autentica tu ambiente de trabajo**

En el caso que estés ejecutando esta libreta en un Google Colab, descomenta la siguiente celda para realizar la autenticación de la sesión en la libreta con Google Cloud. Este paso es importante **para la utilización en Colab**, así podemos garantizar que las llamadas a las APIs de Google Cloud funcionen sin problemas.

In [ ]:
"""
import sys
# Autenticacion adicional se necesita en Google Colab
if "google.colab" in sys.modules:
    # Autentica el usuario con Google Cloud
    from google.colab import auth

    auth.authenticate_user()
"""

### **Sólo para uso en Colab - define el proyecto en Google Cloud a utilizar**

En el caso que estés ejecutando esta libreta en Google Colab, descomenta la siguiente celda para definir qué proyecto en Google Cloud será usado por Colab en la ejecución de esta libreta. De lo contrario, continua con los siguientes pasos.

In [ ]:
"""
if "google.colab" in sys.modules:
    # Define la información del proyecto
    PROJECT_ID = "your-project-id" # @param {type:"string"}
    LOCATION = "us-central1" # @param {type:"string"}

    # Inicializa Vertex AI
    import vertexai

    vertexai.init(project=PROJECT_ID, location=LOCATION)
"""

### **Sólo para uso en entorno de desarrollo local - configurar las credenciales de aplicación por defecto (ADC)**

<div class="alert alert-block alert-warning">
  <strong>
    ⚠️ Importante:
  </strong> Este proceso se debe realizar sólo una vez. Si ejecutaste
  una libreta con anterioridad, no es necesario ejecutarlo de nuevo.
</div>

En el caso que estés ejecutando esta libreta en un entorno de desarrollo local, necesitarás configurar las credenciales de aplicaciones por defecto (ADC). ADC es una estrategia usada por las librerías de Google para automáticamente encontrar credenciales de acceso basadas en el entorno de la aplicación. Para ello, sigue los siguientes pasos:

- [Instala y configura la herramienta gcloud CLI](https://cloud.google.com/sdk/docs/install)
- Crea el archivo de credenciales

In [ ]:
#! gcloud auth application-default login

### Importa las bibliotecas necesarias

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.simplefilter('ignore', UserWarning)

from vertexai.generative_models import (
  GenerationConfig,
  GenerativeModel,
  Image,
  Part
)

## Importa el modelo `Gemini 1.5 Pro Vision`

Gemini Pro Vision (`gemini-1.5-pro-vision`) es un modelo multimodal que admite indicaciones multimodales. Puede incluir texto, imágenes y videos en sus solicitudes de avisos y obtener respuestas en texto o código.

In [3]:
multi_model = GenerativeModel('gemini-1.5-pro-vision')

### Define algunas funciones auxiliares

Define algunas funciones auxiliares. Estas funciones **no** son necesarias para trabajar con Gemini Pro Vision, las usamos para visualizar las imágenes y videos que mandamos como parte de la entrada aquí en la libreta de Python.

In [4]:
import http.client
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps

def display_images(
  images: typing.Iterable[Image],
  max_width: int = 600,
  max_height: int = 350
) -> None:
  for image in images:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)

    if pil_image.mode != 'RGB':
      # RGB es soportado por todos los entornos Jupyter
      pil_image = pil_image.convert('RGB')

    image_width, image_height = pil_image.size

    if max_width < image_width or max_height < image_height:
      # Redimensiona para mostrar la imagen más pequeña en la libreta
      pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))

    IPython.display.display(pil_image)


  def get_image_bytes_from_url (image_url: str) -> bytes:
    with urllib.request.urlopen(image_url) as response:
      response = typing.cast(http.client.HTTPResponse, response)
      image_bytes = response.read()

    return image_bytes


  def load_image_from_url (image_url: str) -> Image:
    image_bytes = get_image_bytes_from_url(image_url)
    return Image.from_bytes(image_bytes)


  def display_content_as_image (content: str | Image | Part) -> bool:
    if not isinstance(content, Image):
      return False

    display_images([ content ])

    return True


  def display_content_as_video (content: str | Image | Part) -> bool:
    if not isinstance(content, Part):
      return False

    part = typing.cast(Part, content)
    file_path = part.file_data.file_uri.removeprefix('gs://')
    video_url = f'https://storage.googleapis.com/{file_path}'

    IPython.display.display(IPython.display.Video(video_url, width=600))

    return True


  def print_multimodal_prompt (contents: list[ str | Image | Part ]):
    for content in contents:
      if display_content_as_image(content):
        continue
      if display_content_as_video(content):
        continue
      
      print(content)
  